# VRP Benchmark Analysis

In [1]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys
import os
import re

# Get the directory of the current notebook
notebook_dir = os.path.dirname(os.path.abspath("__file__"))
# Set the working directory to the project root (one level up from the notebook directory)
analysis_dir = os.path.join(notebook_dir, os.pardir)
project_root = os.path.join(analysis_dir, os.pardir)
os.chdir(project_root)

# You can verify the new working directory
print(f"Current Working Directory: {os.getcwd()}")

from notebooks.analysis.common_analysis_functions import (
    load_benchmark_data, parse_json_columns, filter_optimal_solutions,
    plot_objective_distribution, plot_solve_time_distribution,
    get_baseline_objective, read_baseline_log
)

# Configuration
VRP_BENCHMARK_FILE = 'benchmark/VRP/vrp_benchmark_10cust_2veh.csv'
VRP_MODEL_FILE_PATH = 'models/VRP/vrp_model.py'
VRP_MODEL_DATA_PATH = 'models/VRP/data/vrp_data_10cust_2veh.json'

# Extract data configuration string (e.g., '10cust_2veh')
data_config_match = re.search(r'vrp_data_([\w_]+)\.json', VRP_MODEL_DATA_PATH)
data_config_str = data_config_match.group(1) if data_config_match else 'default'

OUTPUT_PLOTS_DIR = Path(f'results/vrp_analysis_plots/{data_config_str}')
OUTPUT_PLOTS_DIR.mkdir(parents=True, exist_ok=True)

BASELINE_LOG_FILEPATH = Path(f'notebooks/baseline_vrp_log_{data_config_str}.csv')

print(f"Current Working Directory: {os.getcwd()}")
print(f"Plots will be saved to: {OUTPUT_PLOTS_DIR}")
print(f"Baseline log will be saved to: {BASELINE_LOG_FILEPATH}")

Current Working Directory: /home/timpi/Projects/thesis/multi_agent_supply_chain_optimization
Current Working Directory: /home/timpi/Projects/thesis/multi_agent_supply_chain_optimization
Plots will be saved to: results/vrp_analysis_plots/10cust_2veh
Baseline log will be saved to: notebooks/baseline_vrp_log_10cust_2veh.csv


## 1. Load and Preprocess Data

In [2]:
vrp_df = load_benchmark_data(VRP_BENCHMARK_FILE)
vrp_df = parse_json_columns(vrp_df, ['parameters', 'constraints', 'variables', 'modification', 'placeholder_values'])

print("\nDataFrame Info:")
vrp_df.info()
print("\nFirst 5 rows:")
print(vrp_df.head())

Successfully loaded data from benchmark/VRP/vrp_benchmark_10cust_2veh.csv. Shape: (1081, 14)

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1081 entries, 0 to 1080
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   run_id                     1081 non-null   object 
 1   timestamp                  1081 non-null   object 
 2   status                     1081 non-null   object 
 3   objective_value            1081 non-null   float64
 4   model_file_path            1081 non-null   object 
 5   model_data_path            1081 non-null   object 
 6   parameters                 1081 non-null   object 
 7   constraints                1081 non-null   object 
 8   variables                  1081 non-null   object 
 9   pulp_model_execution_time  1081 non-null   float64
 10  modification               1081 non-null   object 
 11  scenario_text              1081 non-null   object 

## 2. Determine Baseline Objective and Calculate Delta Objective

In [3]:
# Get baseline objective
baseline_obj_value = get_baseline_objective(VRP_MODEL_FILE_PATH, VRP_MODEL_DATA_PATH, baseline_log_filepath=str(BASELINE_LOG_FILEPATH))

if baseline_obj_value is not None:
    # Calculate delta_obj for optimal solutions
    vrp_df['delta_obj'] = vrp_df.apply(
        lambda row: row['objective_value'] - baseline_obj_value if row['status'] == 'Optimal' else None,
        axis=1
    )
    print(f"Baseline objective value: {baseline_obj_value}")
    print("Delta objective calculated.")
else:
    print("Warning: Baseline objective not available. Delta objectives not calculated.")
    vrp_df['delta_obj'] = None

Running baseline model from models/VRP/vrp_model.py with data models/VRP/data/vrp_data_10cust_2veh.json...

log - Running optimization model...

log - Executing model source code...

log - Model execution completed.

log - Extracting optimization results...

log - Optimization results extracted.
Logging run 'baseline_run' to 'notebooks/baseline_vrp_log_10cust_2veh.csv'...
Successfully logged 1 runs to notebooks/baseline_vrp_log_10cust_2veh.csv

log - Optimization Completed.
Baseline objective value: 330.1
Baseline objective value: 330.1
Delta objective calculated.


## 3. Overall Performance Analysis

In [4]:
# Filter for optimal solutions for objective/solve time analysis
optimal_vrp_df = filter_optimal_solutions(vrp_df)

if not optimal_vrp_df.empty:
    print("\nOptimal Solutions Summary:")
    print(optimal_vrp_df[['objective_value', 'pulp_model_execution_time', 'delta_obj']].describe())

    plot_objective_distribution(optimal_vrp_df, title='VRP Objective Value Distribution (Optimal Solutions)',
                                save_path=OUTPUT_PLOTS_DIR / 'vrp_objective_distribution.png')
    plot_solve_time_distribution(optimal_vrp_df, title='VRP Solve Time Distribution (Optimal Solutions)',
                                 save_path=OUTPUT_PLOTS_DIR / 'vrp_solve_time_distribution.png')
    
    # Plotting Delta Objective Value Distribution
    if 'delta_obj' in optimal_vrp_df.columns and optimal_vrp_df['delta_obj'].notna().any():
        plt.figure(figsize=(10, 6))
        sns.histplot(optimal_vrp_df['delta_obj'].dropna(), kde=True, bins=30)
        plt.title('Distribution of VRP Delta Objective Values (Optimal Solutions)')
        plt.xlabel('Delta Objective Value')
        plt.ylabel('Frequency')
        plt.grid(True)
        plt.savefig(OUTPUT_PLOTS_DIR / 'vrp_delta_objective_distribution.png')
        plt.close()
    else:
        print("No valid delta objective values to plot.")

else:
    print("No optimal solutions found in the benchmark data for overall performance analysis.")


Optimal Solutions Summary:
       objective_value  pulp_model_execution_time     delta_obj
count      1077.000000                1077.000000  1.077000e+03
mean        346.422154                   0.006333  1.632215e+01
std          28.173691                   0.001525  2.817369e+01
min         296.260000                   0.003186 -3.384000e+01
25%         330.100000                   0.005781  0.000000e+00
50%         330.100000                   0.006743  5.684342e-14
75%         362.570000                   0.007121  3.247000e+01
max         435.180000                   0.017859  1.050800e+02


## 4. Scenario-Specific Analysis: Effects of Modifications

In [5]:
print("\nUnique Scenario Types:")
unique_scenario_types = vrp_df['scenario_type'].unique()
print(unique_scenario_types)

def plot_scenario_impact(df, scenario_type, x_param_key, x_label, title, plot_filename, baseline_value=None):
    """
    Generates a plot for a specific scenario type showing impact on objective value.
    """
    scenario_df = df[df['scenario_type'] == scenario_type].copy()
    scenario_df['varying_param'] = scenario_df['placeholder_values'].apply(lambda x: x.get(x_param_key) if isinstance(x, dict) else None)
    
    scenario_df_filtered = scenario_df.dropna(subset=['varying_param', 'objective_value']).sort_values(by='varying_param')

    if not scenario_df_filtered.empty:
        plt.figure(figsize=(12, 7))
        sns.lineplot(data=scenario_df_filtered, x='varying_param', y='objective_value', marker='o')
        
        if baseline_value is not None:
            plt.axhline(baseline_value, color='red', linestyle='--', label=f'Baseline Objective: {baseline_value:.2f}')
            plt.legend(loc='upper right')

        plt.title(title)
        plt.xlabel(x_label)
        plt.ylabel('Objective Value')
        plt.grid(True)
        plot_path = OUTPUT_PLOTS_DIR / plot_filename
        plt.savefig(plot_path)
        plt.close()
        print(f"Plot saved: {plot_path}")
    else:
        print(f"No valid data to plot for {scenario_type}.")

# --- Individual Scenario Plots ---

# 4.1 Demand Increase - Customer Percentage
plot_scenario_impact(
    vrp_df,
    scenario_type='demand-increase-customer-pct',
    x_param_key='P',
    x_label='Percentage Increase in Customer Demand (in %)',
    title='Impact of Customer Demand Percentage Increase on Objective Value',
    plot_filename='vrp_demand_increase_customer_pct_impact.png',
    baseline_value=baseline_obj_value
)

# 4.2 Demand Increase - Customer Integer
plot_scenario_impact(
    vrp_df,
    scenario_type='demand-increase-customer-int',
    x_param_key='V',
    x_label='New Customer Demand Value (in units)',
    title='Impact of Specific Customer Demand Value on Objective Value',
    plot_filename='vrp_demand_increase_customer_int_impact.png',
    baseline_value=baseline_obj_value
)

# 4.3 Demand Increase - All Customers
plot_scenario_impact(
    vrp_df,
    scenario_type='demand-increase-all',
    x_param_key='P',
    x_label='Percentage Increase in All Demands (in %)',
    title='Impact of Overall Demand Increase on Objective Value',
    plot_filename='vrp_demand_increase_all_impact.png',
    baseline_value=baseline_obj_value
)

# 4.4 Capacity Change
plot_scenario_impact(
    vrp_df,
    scenario_type='capacity-change',
    x_param_key='CAP',
    x_label='Vehicle Capacity (in units)',
    title='Impact of Vehicle Capacity Change on Objective Value',
    plot_filename='vrp_capacity_change_impact.png',
    baseline_value=baseline_obj_value
)

# 4.5 Fleet Size Change
plot_scenario_impact(
    vrp_df,
    scenario_type='fleetsize-change',
    x_param_key='KNEW',
    x_label='Number of Vehicles',
    title='Impact of Fleet Size Change on Objective Value',
    plot_filename='vrp_fleetsize_change_impact.png',
    baseline_value=baseline_obj_value
)

# 4.6 Forbid Arc (I, J)
# Forbid/Force arc scenarios involve two varying parameters (I and J).
# Plotting against a single numerical parameter might not be meaningful for trends.
# Instead, we can show a distribution or count of affected scenarios.
# For now, we'll plot against 'I' as a placeholder, but this might need a different visualization type.
plot_scenario_impact(
    vrp_df,
    scenario_type='forbid-arc',
    x_param_key='I',
    x_label='Node I Index (forbid arc)',
    title='Impact of Forbidding Specific Arc (I,J) on Objective Value',
    plot_filename='vrp_forbid_arc_impact.png',
    baseline_value=baseline_obj_value
)

# 4.7 Force Arc (I, J)
plot_scenario_impact(
    vrp_df,
    scenario_type='force-arc',
    x_param_key='I',
    x_label='Node I Index (force arc)',
    title='Impact of Forcing Specific Arc (I,J) on Objective Value',
    plot_filename='vrp_force_arc_impact.png',
    baseline_value=baseline_obj_value
)


Unique Scenario Types:
['demand-increase-customer-pct' 'demand-increase-customer-int'
 'demand-increase-all' 'capacity-change' 'fleetsize-change' 'forbid-arc'
 'force-arc']
Plot saved: results/vrp_analysis_plots/10cust_2veh/vrp_demand_increase_customer_pct_impact.png
Plot saved: results/vrp_analysis_plots/10cust_2veh/vrp_demand_increase_customer_int_impact.png
Plot saved: results/vrp_analysis_plots/10cust_2veh/vrp_demand_increase_all_impact.png
Plot saved: results/vrp_analysis_plots/10cust_2veh/vrp_capacity_change_impact.png
Plot saved: results/vrp_analysis_plots/10cust_2veh/vrp_fleetsize_change_impact.png
Plot saved: results/vrp_analysis_plots/10cust_2veh/vrp_forbid_arc_impact.png
Plot saved: results/vrp_analysis_plots/10cust_2veh/vrp_force_arc_impact.png


## 5. Model-Specific Analysis: Route Analysis

In [6]:
def extract_routes(variables_dict, num_nodes):
    routes = []
    # Assuming 'x_i_j_k' variables for flow from i to j by vehicle k
    # This is a simplified example and might need adjustment based on actual VRP model variable naming
    
    # For simplicity, let's just count active arcs for now
    active_arcs = []
    for var_name, var_value in variables_dict.items():
        if var_name.startswith('x_') and var_value > 0.5:
            try:
                parts = var_name.split('_')
                i = int(parts[1])
                j = int(parts[2])
                # k = int(parts[3]) # Vehicle index if needed
                active_arcs.append((i, j))
            except (ValueError, IndexError):
                continue
    return active_arcs

# Example usage (requires 'variables' column to be parsed JSON)
# This part needs actual VRP model variable structure to be precise
# For now, just demonstrate how to access and potentially process 'variables'
print("\nExample: Accessing variables for route analysis (requires specific VRP model variable structure):")
if not optimal_vrp_df.empty:
    sample_run = optimal_vrp_df.iloc[0]
    if 'variables' in sample_run and isinstance(sample_run['variables'], dict):
        # Assuming 'num_nodes' can be derived from parameters or data
        # For VRP, num_nodes is len(data['distance'])
        # You would need to pass the original data or derive num_nodes from parameters
        # For this example, let's assume a fixed number of nodes for demonstration
        num_nodes_example = 10 # Placeholder, replace with actual logic
        active_arcs = extract_routes(sample_run['variables'], num_nodes_example)
        print(f"Active arcs in a sample optimal run: {active_arcs[:5]}...")
        print("Variables column contains dictionary. Further VRP-specific route analysis can be implemented here.")
    else:
        print("Variables column is not in expected dictionary format or not present.")
else:
    print("No optimal VRP solutions to analyze routes.")


Example: Accessing variables for route analysis (requires specific VRP model variable structure):
Active arcs in a sample optimal run: [(0, 4), (0, 5), (10, 9), (1, 6), (2, 8)]...
Variables column contains dictionary. Further VRP-specific route analysis can be implemented here.


## 6. Conclusion

In [7]:
print("VRP benchmark analysis complete. Plots saved to:", OUTPUT_PLOTS_DIR)

VRP benchmark analysis complete. Plots saved to: results/vrp_analysis_plots/10cust_2veh
